In [9]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import lmfit as lm
import sys
sys.path.append('../../python')
import dataPython as dp
import scipy.interpolate as inter

from datetime import datetime
import scipy.integrate as si
import noordermeer as noord
import time

ModuleNotFoundError: No module named 'dataPython'

In [10]:
'''
source [1]: 
https://www.aanda.org/articles/aa/pdf/2011/07/aa16634-11.pdf
run in gipsy
Zlaw = sech-squared ???
Z0=.88 kpc (paper table 4 gives hz=.44kpc where z0=2*hz)
D=14.6 Mpc (paper table 1)
density profile Y
UNITS=kpc,mag/arcsec**2
RADDENS=<NGC7814disk4  <-- note that gipsy doesn't like scientific notation, i.e. 1.534e2 should be 153.4
MASS=1 ???? (stellar mass....)
RADII=0,0.64,1.74,2.83,3.96,5.10,6.23,7.36,8.49,9.63,10.76,11.89,13.02,14.16,15.29,16.42,17.55,18.69,19.82
(same points as raw data, table 3)
'''

#************************automatically replacing '!' with '#'
import fileinput
with fileinput.FileInput('../../testing/7814reallybulge.dat', inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace('!','#'), end=' ')
    
with fileinput.FileInput('../../testing7814reallydisk.dat', inplace=True, backup= '.bak') as file:
    for line in file:
        print(line.replace('!','#'), end=' ')
        
#importing text files
#there's no need to import the radius for each component as everything has the same r array (the r array of the raw data)
#data:

data = dp.getXYZdata_wXYerr('ngc7814data')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err1 = np.asarray(data['ey'])

#disk:
disk = dp.getXYZdata('../../testing/7814reallydisk.dat')
d = np.asarray(disk['zz'])

#bulge:
bulge.dp.getXYZdata('../../testing/7814really bulge.dat')
b = np.asarray(bulge['zz'])


#define total curve
#D=9.25 #disk M-L ration provided in [1]
#B=.5 #bulge M-L ration provided in [1]
def t(B,D):
    return np.sqrt(((D*d)**2)+(B*b)**2)

plt.figure(figsize=(12,8))
plt.plot(r,d,label = 'Diskunfitted')
'''
GIPSY input:
disk
Zlaw = sech-squared
Z0=.88
UNITS=kpc,msun/pc**2
RADDENS=<7814diskdecomp4
MASS=1
RADII=0.64,1.74,2.83,3.96,5.10,6.23,7.36,8.49,9.63,10.76,11.89,13.02,14.16,15.29,16.42,17.55,18.69,19.82
'''
plt.plot(r,b,':',label='Bulge unfitted')
'''
GIPSY input:
bulge
UNITS=kpc,msun/pc**2
RADDENS=<7814bulgedecomp3 
MASS=1
RADII=0.64,1.74,2.83,3.96,5.10,6.23,7.36,8.49,9.63,10.76,11.89,13.02,14.16,15.29,16.42,17.55,18.69,19.82
'''
plt.plot(brb,t(1,1), label='total unfitted')
#where did the brb come from?
plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo', label='Data')

plt.xlabel('r (kpc)')
plt.ylabel('v (km/s)')
plt.legend()
plt.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../testing/7814reallybulge.dat' -> '../../testing/7814reallybulge.dat.bak'

In [8]:
# https://iopscience.iop.org/article/10.3847/0004-6256/152/6/157 fig 5

fromplot_import = dp.getXYdata('NGC7814photometric.rcl')
r_fromplot = fromplot_import['xx']
v_fromplot = fromplot_import['yy']
v_fromplotZ = np.array(v_fromplot)

D=14.6e3 #kpc
r_fromplotZ = D*np.tan(np.radians(np.array(r_fromplot)/3600))
# Double check what is going on here^^
#convert angular distance (arcsec) to linear distance (kpc)

dat= np/array([r_fromplotZ, v_fromplotZ])
dat = dat.T
# where did T come from?
#np.savetxt('data.rcl', dat, delimiter = '         ')
rrrr = np.array([r_fromplotZ]).T

plt.figure(figsize=(6,4))
r_fromplot2 = r_fromplot[44:]
v_fromplot2 = v_fromplot[44:]
# I don't know what is happening here

m, b = no.polyfit(r_fromflot2,v_fromplot2, 1)
# ^^What? related to def below

def y(x):
    return(m*x+b)
x0=r_fromplot[-1:] #the very last element of radius array, to be used as last element in new radius array
x=np.linspace(x,x0,100)
#turn x and y column vectors into (horizontal) arrays
y=y(x).T
x=x.T
# ^^This may be what the .T is doing

#remove the redundant bracket by calling the "nested" bracket
#what is the redundant bracket and why are we removing it?
y=y[0]
x=x[0]
dat = np.array([x,y])
dat = dat.T
#np.savetxt('')

plt.plot(r_fromplotZ,v_fromplot, ':'. color = 'm', markersize=10,label='photometric profile from sparc in kpc')

plt.xlim([0,23])
plt.ylim([10,22])
plt.gca().invery_yaxis()
## ^^ ??
plt.legend()
plt.show()



SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-8-5eeb3d0c60cd>, line 44)

In [ ]:
'''
SEPARATING SPARC DECOMP INTO JUST DISK AND JUST BULGE... FOR GIPYSY...
note that for some reason the files must be saved in the testing folder, otherwise I can't push the files to Kitty...
'''
decomp = dp.getXYZdata('../NGC7814_decomp.dens')
r_decomp = decomp['xx'] #units kps
d_decomp = decomp['yy'] #Lsun/pc**2
b_decomp = decomp['zz'] #lsun/pc**2

#gipsy doesn't like any values >=0 (<=0??), so remove first element
del r_decomp[0]
del d_decomp[0]
del b_decomp[0]
#not: gipsy doesn't like more than 3 decimals. gipsy also doesn't like values of zero (ie the first point in the data file)
dfile = np.array([r_decomp, d_decomp])
dfile = dfile.T
#np.save
'''
Gipsy input:
'''

bfile = np.array([r_decomp, b_decomp])
bfile = bfile.T
#np.savetxt